In [24]:
import os
import pandas as pd
import numpy as np
import copy
from openpyxl import load_workbook
    
cwd = os.getcwd()
cwd = os.chdir('C:\Users\jojoseph\Desktop\Python Models\Python\IO')

print (os.getcwd())

#jj we can pprobably read the sheet names from the workbook
#define a way to call the number of regions to be analyzed
all_reg_names=['region1','region2']
all_reg_values=[1,2]

C:\Users\jojoseph\Desktop\Python Models\Python\IO


In [25]:
#opening the social account matrix excel files from IMPLAN -(89 sectors summarized from study area data/study area explorer )
new_MRIO= 'new_MRIO.xlsx'# open file with social account matrices
reg_soc_acc_mat=pd.ExcelFile(new_MRIO)
print (reg_soc_acc_mat.sheet_names)

#region3=reg_soc_acc_mat.parse('region2')

[u'region1', u'region2']


In [33]:
#opening the required files using while loop
dict_of_df = {}
count=1
for x in all_reg_names:
    
    df = reg_soc_acc_mat.parse(x)  # sheet name
    key_name = 'region'+str(count) 
    dict_of_df[key_name] = copy.deepcopy(df)
    count=count+1

dict_of_df.keys()
dict_of_df

{'region1':      industry_code                                        description  \
 0                0                                              total   
 1                1                                    oilseed farming   
 2                2                                      grain farming   
 3                3                        vegetable and melon farming   
 4                4                                      fruit farming   
 5                5                                   tree nut farming   
 6                6   greenhouse, nursery, and floriculture production   
 7                7                                    tobacco farming   
 8                8                                     cotton farming   
 9                9                   sugarcane and sugar beet farming   
 10              10                             all other crop farming   
 11              11  beef cattle ranching and farming, including fe...   
 12              12        

In [35]:
for x in all_reg_names:
    print ( dict_of_df[x].iloc[0:5,0:1])


   industry_code
0              0
1              1
2              2
3              3
4              4
   industry_code
0              0
1              1
2              2
3              3
4              4
